# Use Case A - Query builder (in progress)
***
### Using 2-grams, 3-grams, 4-grams with content from the Glossary articles and the Statistics Explained articles


In [1]:


import re

import pandas as pd
import numpy as np

import pyodbc

import gensim

from datetime import datetime

import unicodedata

pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Load data from the database

In [2]:
c = pyodbc.connect('DSN=Virtuoso All;DBA=ESTAT;UID=user_name;PWD=password')
cursor = c.cursor()

In [3]:
for row in cursor.tables():
    print (row.table_name)

dat_article
dat_article_paragraph
dat_article_shared_link
dat_further_info
dat_glossary
dat_link_info
dat_paragraph_figure
dat_redirections
dat_related_concepts
dat_sources
dat_statistical_data
mod_article_division
mod_resource_information
mod_resource_type
tm_articles
tm_terms
tm_topics
tm_values


### Glossary articles  

* Definitions and last update from dat_glossary.
* Titles and URLs from dat_link_info (with resource_information_id=1).

In [4]:
SQLCommand = """SELECT T1.id, T1.definition, T1.last_update, T2.title, T2.url 
                FROM ESTAT.V1.dat_glossary as T1 
                INNER JOIN ESTAT.V1.dat_link_info as T2  
                  ON T1.id=T2.id 
                WHERE T2.resource_information_id=1 """

GL_df = pd.read_sql(SQLCommand,c)
GL_df = GL_df[['id', 'title', 'definition', 'url','last_update']]
GL_df.head()

,id,title,definition,url,last_update
0,516,Farm structure survey (FSS) ? glossary,"The basic Farm structure survey , abbreviated as FSS and also known as Survey on the structure of agricultural holdings , is carried out by all European Union (EU) Member States. The FSS is conducted consistently throughout the EU with a common methodology on a regular basis and provides therefore comparable and representative statistics across countries and time, at regional levels (down to NUTS 3 level). Every 3 or 4 years the FSS is carried out as a sample survey, and once in ten years as a census . The 2000 census covered the 15 Member States at that time, plus Latvia, Hungary, Slovenia, Slovakia and Norway, while the 2010 census covered the 27 Member States at that time, plus Croatia, Iceland, Norway, Switzerland, Montenegro and Serbia. In 2010 a special survey, the Survey on agricultural production methods (SAPM) was carried out. SAPM was carried out together with the 2010 census in some countries, whereas in other countries SAPM was carried out as a sample survey and data were linked to data of the census at the level of the individual holding to enable cross comparisons of variables collected in both SAPM and the census. Sample surveys of 2003, 2005 and 2007 covered the EU-27 Member States. In 2013 and 2016, the survey covered the EU-28 Member States. Iceland, Norway, Switzerland, Montenegro, North Macedonia and Serbia also send farm structure data to Eurostat. The countries collect information from individual agricultural holdings and, observing strict rules of confidentiality, data are forwarded to Eurostat . The information collected in the FSS covered land use , livestock numbers, rural development, management and farm labour input (including the age, gender and relationship to the holder of the agricultural holding). The survey data can then be aggregated by different geographic levels (countries, regions, and for basic surveys also district level). The data can also be arranged by size class, area status, legal status of the holding, objective zone and farm type. The basic unit underlying the FSS is the agricultural holding. The FSS covered all agricultural holdings which meet the minimum requirements set in the applicable legislation, see also the article on Farm structure survey - thresholds . The legal basis for the FSS was Regulation (EC) No 1166/2008 of 19 November 2008 on farm structure surveys and the survey on agricultural production methods, which repealed Council Regulation (EEC) No 571/88 . The Regulation (EC) No 1166/2008 on farm structure surveys ceased to provide statistical information as of 2018. It was repealed by Regulation (EC) No 2018/1091 .",https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Glossary:Farm_structure_survey_(FSS),2019-12-03 18:07:00
1,517,Standard Output ? glossary,"The standard output of an agricultural product (crop or livestock), abbreviated as SO , is the average monetary value of the agricultural output at farm-gate price, in euro per hectare or per head of livestock. There is a regional SO coefficient for each product, as an average value over a reference period (5 years, except for the SO 2004 coefficient calculated using the average of 3 years). The sum of all the SO per hectare of crop and per head of livestock in a farm is a measure of its overall economic size , expressed in euro. The standard output is used to classify agricultural holdings by type of farming and by economic size. The common classification of holdings is built on different nested levels. Homogeneous groups of holdings can be assembled in a greater or lower degree of aggregation. Depending on the amount of details required, the types of farming are divided into : general types of farming (1 digit), principal types of farming (2 digits), particular types of farming (3 digits), in earlier years in a few cases split into subdivisions of particular types of farming (4 digits). Table 1 shows the changes in t

In [5]:
GL_df = GL_df.replace('', np.nan) 
print(GL_df.isnull().sum())


id             0
title          0
definition     2
url            0
last_update    0
dtype: int64


### Delete records with empty definitions and data cleansing

* Also put the URLs in lists

In [6]:
## drop records with any field empty
GL_df = GL_df.dropna(axis=0,how='any')
print(GL_df.isnull().sum())
GL_df.reset_index(inplace=True)

import re
import unicodedata as ud

def clean(x, quotes=True):
    if pd.isnull(x): return x  
    x = x.strip()
    
    ## make letter-question mark-letter -> letter-quote-space-letter !!! but NOT in the lists of URLs!!!
    if quotes:
        x = re.sub(r'([A-Za-z])\?([A-Za-z])','\\1\' \\2',x) ## NEW
    
    ## make letter-question mark-space lower case letter letter-quote-space letter
    x = re.sub(r'([A-Za-z])\? ([a-z])','\\1\' \\2',x) ## NEW

    ## delete ,000 commas in numbers    
    x = re.sub(r'\b(\d+),(\d+)\b','\\1\\2',x) ## CORRECTED
    
    ## delete  000 spaces in numbers
    x = re.sub(r'\b(\d+) (\d+)\b','\\1\\2',x) ## CORRECTED
    
    ## remove more than one spaces
    x = re.sub(r' +', ' ',x)
    
    ## remove start and end spaces
    x = re.sub(r'^ +| +$', '',x,flags=re.MULTILINE) 
    
    ## space-comma -> comma
    x = re.sub(r' \,',',',x)
    
    ## space-dot -> dot
    x = re.sub(r' \.','.',x)
    
    #x = x.encode('latin1').decode('utf-8') ## â\x80\x99
    x = ud.normalize('NFKD',x).encode('ascii', 'ignore').decode()
    
    return x


GL_df['title'] = GL_df['title'].apply(clean)
GL_df['title'] = GL_df['title'].apply(lambda x: re.sub(r'\?','-',x))
GL_df['definition'] = GL_df['definition'].apply(clean)
GL_df['url'] = GL_df['url'].apply(clean,quotes=False)
GL_df['url'] = GL_df['url'].apply(lambda x: [x])

GL_df.head()

id             0
title          0
definition     0
url            0
last_update    0
dtype: int64


,index,id,title,definition,url,last_update
0,0,516,Farm structure survey (FSS) - glossary,"The basic Farm structure survey, abbreviated as FSS and also known as Survey on the structure of agricultural holdings, is carried out by all European Union (EU) Member States. The FSS is conducted consistently throughout the EU with a common methodology on a regular basis and provides therefore comparable and representative statistics across countries and time, at regional levels (down to NUTS 3 level). Every 3 or 4 years the FSS is carried out as a sample survey, and once in ten years as a census. The 2000 census covered the 15 Member States at that time, plus Latvia, Hungary, Slovenia, Slovakia and Norway, while the 2010 census covered the 27 Member States at that time, plus Croatia, Iceland, Norway, Switzerland, Montenegro and Serbia. In 2010 a special survey, the Survey on agricultural production methods (SAPM) was carried out. SAPM was carried out together with the 2010 census in some countries, whereas in other countries SAPM was carried out as a sample survey and data were linked to data of the census at the level of the individual holding to enable cross comparisons of variables collected in both SAPM and the census. Sample surveys of 2003, 2005 and 2007 covered the EU-27 Member States. In 2013 and 2016, the survey covered the EU-28 Member States. Iceland, Norway, Switzerland, Montenegro, North Macedonia and Serbia also send farm structure data to Eurostat. The countries collect information from individual agricultural holdings and, observing strict rules of confidentiality, data are forwarded to Eurostat. The information collected in the FSS covered land use, livestock numbers, rural development, management and farm labour input (including the age, gender and relationship to the holder of the agricultural holding). The survey data can then be aggregated by different geographic levels (countries, regions, and for basic surveys also district level). The data can also be arranged by size class, area status, legal status of the holding, objective zone and farm type. The basic unit underlying the FSS is the agricultural holding. The FSS covered all agricultural holdings which meet the minimum requirements set in the applicable legislation, see also the article on Farm structure survey - thresholds. The legal basis for the FSS was Regulation (EC) No 1166/2008 of 19 November 2008 on farm structure surveys and the survey on agricultural production methods, which repealed Council Regulation (EEC) No 571/88. The Regulation (EC) No 1166/2008 on farm structure surveys ceased to provide statistical information as of 2018. It was repealed by Regulation (EC) No 2018/1091.",[https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Glossary:Farm_structure_survey_(FSS)],2019-12-03 18:07:00
1,1,517,Standard Output - glossary,"The standard output of an agricultural product (crop or livestock), abbreviated as SO, is the average monetary value of the agricultural output at farm-gate price, in euro per hectare or per head of livestock. There is a regional SO coefficient for each product, as an average value over a reference period (5 years, except for the SO 2004 coefficient calculated using the average of 3 years). The sum of all the SO per hectare of crop and per head of livestock in a farm is a measure of its overall economic size, expressed in euro. The standard output is used to classify agricultural holdings by type of farming and by economic size. The common classification of holdings is built on different nested levels. Homogeneous groups of holdings can be assembled in a greater or lower degree of aggregation. Depending on the amount of details required, the types of farming are divided into : general types of farming (1 digit), principal types of farming (2 digits), particular types of farming (3 digits), in earlier years in a few cases split into subdivisions of particular types of farming (4 digits). Table 1 shows the changes in

### Delete "special" records

In [7]:
## Drop The records with definitions "The revision ..." and "Redirect to ..." 

idx = GL_df[GL_df['definition'].str.startswith('The revision #')].index
print(idx)
GL_df.drop(idx , inplace=True)
idx = GL_df[GL_df['definition'].str.startswith('Redirect to')].index
print(idx)
GL_df.drop(idx , inplace=True)
GL_df.reset_index(drop=True, inplace=True)

Int64Index([ 220,  231,  316,  351,  460,  486,  491,  494,  499,  550,  561,
             579,  597,  635,  722,  763,  781,  807,  843,  914,  963,  965,
             974,  999, 1005, 1124, 1135, 1180, 1230],
           dtype='int64')
Int64Index([], dtype='int64')


### Check for duplicate titles AND definitions

In [8]:
idx = GL_df[GL_df.duplicated(subset=['title','definition'], keep=False)].sort_values(by=['title','definition']).index
print(idx)
GL_df.drop(idx , inplace=True)
GL_df.reset_index(drop=True, inplace=True)

Int64Index([1217, 1219], dtype='int64')


### Also produce processed file for debugging

In [9]:
## !!! below only for debugging
def file_name(pre,ext):
    current_time = datetime.now() 
    return pre + '_'+ str(current_time.month)+ '_' + str(current_time.day) + \
                 '_' + str(current_time.hour)+ '_' + str(current_time.minute)  +'.'+ext
    
outfile = file_name('D3_2_Glossary','xlsx')
GL_df.to_excel(outfile)


### Statistics explained articles

* Ids, titles and urls from dat_article and dat_link_info (with resource_information_id=1).
* Also put the URLs in lists

In [10]:
SQLCommand = """SELECT T1.id, T2.title, T2.url 
                FROM ESTAT.V1.dat_article as T1 
                INNER JOIN ESTAT.V1.dat_link_info as T2  
                  ON T1.id=T2.id 
                WHERE T2.resource_information_id=1 """

SE_df = pd.read_sql(SQLCommand,c)

SE_df['title'] = SE_df['title'].apply(clean)
SE_df['url'] = SE_df['url'].apply(clean,quotes=False)
SE_df['url'] = SE_df['url'].apply(lambda x: [x])
SE_df.head()

,id,title,url
0,1,Accessing European statistics,[https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Accessing_European_statistics]
1,8,Adult Education Survey (AES) methodology,[https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Adult_Education_Survey_(AES)_methodology]
2,13,Adult learning statistics,[https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Adult_learning_statistics]
3,14,Adult learning statistics - characteristics of education and training,[https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Adult_learning_statistics_-_characteristics_of_education_and_training]
4,17,Agri-environmental indicator - High Nature Value farmland,[https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Agri-environmental_indicator_-_High_Nature_Value_farmland]


### Add content

* From dat_article_paragraph with abstract=0.

In [11]:
SQLCommand = """SELECT T3.article_id, T3.title, T3.content 
                FROM ESTAT.V1.dat_article as T1 
                INNER JOIN ESTAT.V1.dat_article_paragraph as T3 
                  ON T1.id=T3.article_id  
                WHERE T3.abstract=0 """

add_content = pd.read_sql(SQLCommand,c)
add_content['title'] = add_content['title'].apply(clean)
add_content['content'] = add_content['content'].apply(clean)
add_content

,article_id,title,content
0,1,Statistical themes,"The information on Eurostatas website under the heading of a Browse statistics by theme a (alternatively, click on aData > Browse statistics by themea in the top menu bar of most webpages on the Eurostat website) provides links to: EU policy indicators (statistics for European policies and high-priority initiatives ); Statistics by theme: General and regional statistics; Economy and finance; Population and social conditions; Industry, trade and services; Agriculture and fisheries; International trade; Transport; Environment and energy; Science, technology and digital society. General and regional statistics; Economy and finance; Population and social conditions; Industry, trade and services; Agriculture and fisheries; International trade; Transport; Environment and energy; Science, technology and digital society. For each of these themes, the user is presented with a range of different subthemes. These subthemes are presented through adedicated sectionsa on each subject area, with links to information for data (main tables and databases), legislation, methodology, publications and other background information."
1,1,Access to data,"Data navigation tree The majority of Eurostatas statistics may be accessed from the data navigation tree ; alternatively, click on aData > Databasea in the top menu bar of most webpages on the Eurostat website. The data navigation tree is based on the nine statistical themes presented above and is collapsible and expandable. It has four main branches: Database by themes which contains the full range of public data available on Eurostatas website. These data are presented in multi-dimensional tables with selection features that allow tailor-made presentations and extractions. The interface for databases is called the Data browser and this provides an intuitive way to select and organise information. Data can be downloaded from the Data browser in various formats (XLS, CSV and TSV). Tables by themes which offers a selection of the most important Eurostat data in a user-friendly way. Techical aspects are the same as for ""Database by themes"". Tables on EU policy which also provides access to pre-defined main tables; these have particular relevance for tracking the progress being made in relation to important policy areas, including: the macroeconomic imbalances procedure indicators ; the euro-indicators / principal European economic indicators a PEEIs ; the Europe 2020 indicators; the circular economy indicators; the sustainable development indicators; the Employment and social policy indicators; and the indicators for the European pillar of social rights. The tools for viewing and extracting data are the same as those described above for tables by themes. The fourth branch was created to present some cross-cutting topics, and currently contains data coming from different data sources on the quality of life, migrant integration and children in migration, economic globalisation indicators, equality (age and gender), quality of employment, agri-environmental indicators, climate change and skills-related statistics. The data navigation tree also has two special branches, where new items or recently updated items (from all databases and main tables) can be displayed according to a number of criteria set by the user. Online data codes Eurostat online data codes, such as tps00001 and nama_gdp_c [1], allow easy access to the most recent data on Eurostatas website; these codes are given as part of the source below each table, figure or map. Users can access the freshest data by typing a standardised hyperlink into a web browser a https:/ /ec.europa.eu/eurostat/product' code=<data_code>&mode=view a where <data_code> is to be replaced by the online data code printed under the table or figure in question. The data are presented either in the TGM or the Data explorer interface. Online data codes can also be fed into the aSearcha function on Eurostatas website, whic

### Aggregate above content  by article id

In [12]:

add_content_grouped = add_content.groupby(['article_id'])['title','content'].aggregate(lambda x: list(x))
add_content_grouped.reset_index(inplace=True)
for i in range(len(add_content_grouped)):
    add_content_grouped.loc[i,'raw content'] = ''
    for (a,b) in zip(add_content_grouped.loc[i,'title'],add_content_grouped.loc[i,'content']):
        add_content_grouped.loc[i,'raw content'] += ' '+a + ' ' + b
add_content_grouped = add_content_grouped[['article_id','raw content']]    

add_content_grouped

<ipython-input-12-4edcea5ac3ee>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  add_content_grouped = add_content.groupby(['article_id'])['title','content'].aggregate(lambda x: list(x))


article_id  \
0             1   
1             8   
2            13   
3            14   
4            17   
..          ...   
860        6169   
861        6188   
862        6191   
863        6194   
864        6199   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

### Merge with main file

In [13]:
SE_df = pd.merge(SE_df,add_content_grouped,left_on='id',right_on='article_id',how='inner')
SE_df.drop(['article_id'],axis=1,inplace=True)

SE_df.head()

id                                                                  title  \
0   1                                          Accessing European statistics   
1   8                               Adult Education Survey (AES) methodology   
2  13                                              Adult learning statistics   
3  14  Adult learning statistics - characteristics of education and training   
4  17              Agri-environmental indicator - High Nature Value farmland   

                                                                                                                                          url  \
0                                          [https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Accessing_European_statistics]   
1                               [https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Adult_Education_Survey_(AES)_methodology]   
2                                              [https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Adult_learning_statistics]   
3  [https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Adult_learning_statistics_-_characteristics_of_education_and_training]   
4              [https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Agri-environmental_indicator_-_High_Nature_Value_farmland]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

### Tokenize, remove stop-words and stem; keep also the original terms

In [14]:

from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import stem_text
from gensim.parsing.porter import PorterStemmer

p = PorterStemmer()

def text_to_words(text,url):
    words = str(gensim.utils.simple_preprocess(text, deacc=True))
    words = remove_stopwords(words) 
    words = gensim.utils.tokenize(words)
        
    ## keep also original token!!! 
    words = [[p.stem(token),token,url] for token in words if len(p.stem(token)) >= 5] ##minimum length = 5 
    yield words        

texts=list()   

for i in range(len(GL_df)):
    texts.extend(text_to_words(GL_df.loc[i,'definition'],GL_df.loc[i,'url']))
    texts.extend(text_to_words(GL_df.loc[i,'title'],GL_df.loc[i,'url'])) 
for i in range(len(SE_df)):    
    texts.extend(text_to_words(SE_df.loc[i,'title'],SE_df.loc[i,'url'])) 
    texts.extend(text_to_words(SE_df.loc[i,'raw content'],SE_df.loc[i,'url'])) 

### Co-occurences: keys in n-grams are (n-1) tuples of stemmed tokens 

* Three dictionaries, for 2-,3-,and 4-grams
* For each key in a dictionary, the value is another (nested) dictionary with the **original terms**, their counts and the relevant URLs.
* Below all three dictionaries are constructed from the sequences of 4-grams.


In [15]:
##%%script false --no-raise-error
## Check also COLLOCATIONS: http://www.nltk.org/howto/collocations.html and http://www.nltk.org/api/nltk.html?highlight=ngram


from nltk import bigrams, trigrams, ngrams
#from collections import Counter, defaultdict


model2=dict()
model3=dict()
model4=dict()

def dict_insert(model,entered,proposed,new_urls_to_check):
    key_1 = model.get(entered)
    if key_1:
        key_2 = key_1.get(proposed)
        if key_2:
            key_2[0] +=1
            existing_urls = key_2[1]
            add_urls = [u for u in new_urls_to_check if u not in existing_urls]
            key_2.extend(add_urls)
        else:    
            key_1[proposed]= [1,new_urls_to_check]
    else:
        model[entered]={proposed:[1,new_urls_to_check]}
    return model            


# Co-occurences

for sentence in texts:
    pairs = [elem for elem in sentence] ## a list of 3-tuples (stemmed term, original term, list of URLs)
    if len(pairs) == 0: continue
    
    for first, second, third, fourth in ngrams(pairs,4): ## quadruplets of 3-tuples (stemmed term, original term, list of URLs)
        first_stem, first_orig, first_url = first
        second_stem, second_orig, second_url = second
        third_stem, third_orig, third_url = third
        fourth_stem, fourth_orig, fourth_url = fourth
       
        model2 = dict_insert(model2, first_stem, second_orig,list(set(first_url).intersection(second_url)))
        model2 = dict_insert(model2, second_stem, third_orig,list(set(second_url).intersection(third_url)))
        model2 = dict_insert(model2, third_stem, fourth_orig,list(set(third_url).intersection(fourth_url)))
        
        model3 = dict_insert(model3,(first_stem,second_stem),third_orig,list(set(first_url).intersection(*[second_url,third_url])))
        model3 = dict_insert(model3,(second_stem,third_stem),fourth_orig,list(set(second_url).intersection(*[third_url,fourth_url])))
        
        model4 = dict_insert(model4,(first_stem, second_stem, third_stem),fourth_orig,fourth_url)
   
        
## Transform counts to probabilities

for w1 in model2.keys():
    ssum = sum(model2[w1][w2][0] for w2 in model2[w1].keys())
    for w2 in model2[w1].keys():
        model2[w1][w2][0] /= ssum

for w1_w2 in model3.keys():
    ssum = sum(model3[w1_w2][w3][0] for w3 in model3[w1_w2].keys())
    for w3 in model3[w1_w2].keys():
        model3[w1_w2][w3][0] /= ssum

for w1_w2_w3 in model4.keys():
    ssum = sum(model4[w1_w2_w3][w4][0] for w4 in model4[w1_w2_w3].keys())
    for w4 in model4[w1_w2_w3].keys():
        model4[w1_w2_w3][w4][0] /= ssum


### Query Builder 1
***

In [16]:
import ipywidgets as widgets
layout = widgets.Layout(width='600px', height='30px')

In [17]:

def change_top_articles( Keywords):

    from operator import itemgetter
    p = PorterStemmer()
    
    last_match = ''
    
    #if not Keywords.endswith(' '):
    #    return None
    
    def test_and_back_step(x):
        mod_index = -1
        models = [model2,model3,model4]
        if len(x)==1: 
            x=x[0] ; model=models[0]
        elif len(x) ==2 or len(x) == 3:
            x=tuple(x) ; mod_index=len(x)-1; model=models[mod_index]
        elif len(x)==4:
            x=tuple(x[:3]) ; model=models[2]
        else:
            return None
        while not models[mod_index].get(x) and mod_index >=0:
            x=x[:-1]
            if len(x)==1 : x=x[0]
            mod_index -=1 ; model=models[mod_index] 
        return (model,x)    
            

    x = Keywords.split() 
    if len(x) ==0: 
        print()
        return
    x = [p.stem(el) for el in x]
    
    
    model,x = test_and_back_step(x)
    if not model.get(x):
        return None
    

    print()
    print('Based on last match: ',x,'\n')
    print('Suggestions, probabilities (in descending order) and relevant URLs: ')
    proposals = sorted([(k,v) for (k,v) in model[x].items()],key=itemgetter(1),reverse=True)
    last_match = x
    for key, value in proposals:
        print()
        print(key,': ',value[0])
        for url in value[1]:
            print(url)
   
    
def query_build1(value):
  style = {'description_width': 'initial'}
    
  Keywords = widgets.Text(
      value=value,
      placeholder='Type something',
      description='Keywords:',
      disabled=False
  )

  ui = widgets.HBox([Keywords])
  out = widgets.interactive_output(change_top_articles, {'Keywords': Keywords})
  display(ui, out)
    
query_build1(value='household')  

Output()